# Basic Rosetta functionality

First things first. Let's initialize PyRosetta and set the constants. In a real-world script, the `input_file` variable should be set through the `argparse` module.

In [3]:
from __future__ import print_function

from pyrosetta import *
from rosetta import *
init(extra_options='-beta_nov15')
input_file = '/work/dadriano/PyRosetta_Teaching/src_pdbs/LPC_dryRun_3lB6A_Mf14aa_order00029_ndx6_c189_2cig12p_0001_-41.8_re1_6_EEHEE_0883_0001_0002.pdb.gz'
#input_file = 'LPC_dryRun_3lB6A_Mf14aa_order00029_ndx6_c189_2cig12p_0001_-41.8_re1_6_EEHEE_0883_0001_0002.pdb.gz'

Found rosetta database at: /work/dadriano/.virtualenvs/may2017_pyrosetta_newDigs/lib/python2.7/site-packages/pyrosetta-4.0-py2.7.egg/database; using it....
PyRosetta-4 2016 [Rosetta 2016 unknown:fdd9e71c3a71b1bbd5e9e6f21327432d10df240c 2017-03-27 11:09:48 -0400] retrieved from: git@github.com:RosettaCommons/main.git
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.



## Scoring a pose

In [4]:
p = pose_from_file(input_file)
sf = get_score_function()
print(sf.get_name())

%psearch sf.*weight*
sf.get_weight?

sf.get_weight(core.scoring.hbond_bb_sc)
sf.set_weight(core.scoring.hbond_bb_sc, 1.2)
sf.get_weight(core.scoring.hbond_bb_sc)
sf(p)


beta_nov15.wts


-423.08568743208014

## Evaluating energies between specific atoms

In [5]:
sf.show(p)

resNo = 4 # let's say I care about residue 4
p.energies().show(resNo)

r1 = p.residue(128)
r2 = p.residue(133)
a1 = r1.atom('N')
a2 = r2.atom('O')
etable_atom_pair_energies(a1, a2, sf)


(0.0, 0.0, 0.0)

## Looking up hydrogen bonds

In [28]:
hbond_set = core.scoring.hbonds.HBondSet()
p.update_residue_neighbors() # not actually necessary in this case bacause we just scored
core.scoring.hbonds.fill_hbond_set(p, False, hbond_set)

hbond_set.show(p) # show all of the hydrogen bonds in the pose
hbond_set.show(p, 128) # show the hydrogen bonds involving residue 128

## Accessing and manipulating FoldTrees

In [30]:
print(p.fold_tree())
p.num_jump()

FOLD_TREE  EDGE 1 109 -1  EDGE 1 110 1  EDGE 110 150 -1 


1

## MoveMap

In [31]:
movemap = MoveMap()
movemap.set_bb(True)
movemap.set_bb(50, False)


## MonteCarlo

In [32]:
kT = 1.
mc = MonteCarlo(p, sf, kT)
mc.boltzmann(p) # call this after modifying a pose

mc.show_scores()
mc.show_counters()
mc.show_state()

## Putting it together

In [36]:
n_moves = 1
small_mover = protocols.simple_moves.SmallMover(movemap, kT, n_moves)
small_mover.angle_max('H', 25)
small_mover.angle_max('E', 25)
small_mover.angle_max('L', 25)

trial_mover = TrialMover(small_mover, mc)

# let's apply this trial mover 100x and see what happens
repeat = 100
for _ in range(repeat):
    trial_mover.apply(p)

mc.show_scores()
mc.show_counters()
mc.show_state()

mc.recover_low(p)


## More flow control movers
What is the difference between `one_trial_mover` and `another_trial_mover` and how do they compare to `trial_mover` being called in a for-loop?

In [37]:
repeat_mover = RepeatMover(small_mover, repeat)

one_trial_mover = TrialMover(repeat_mover, mc)

another_repeat_mover = RepeatMover(trial_mover, repeat)
another_trial_mover = TrialMover(repeat_mover, mc)

